In [0]:
from pyspark.sql.functions import current_timestamp,to_utc_timestamp,col

In [0]:
abfss = spark.conf.get('spark.saneamento.datalake.default')

In [0]:
dbutils.widgets.text("databricks_table", "std_san_ligacaoresumocategoria")
dbutils.widgets.text("data_lake_raw_path", "raw/san/ligacaoresumocategoria/2025/11/06")
dbutils.widgets.text("data_lake_std_path", "standardized/san/ligacaoresumocategoria")

dbTable = dbutils.widgets.get("databricks_table")
data_lake_raw_path = dbutils.widgets.get("data_lake_raw_path")
data_lake_std_path = dbutils.widgets.get("data_lake_std_path")

raw_path = f"{abfss}/{data_lake_raw_path}"
std_path = f"{abfss}/{data_lake_std_path}"


In [0]:
try:
  dbutils.fs.ls(raw_path)
except:
  dbutils.notebook.exit("Encerrado por falta de arqivos")


In [0]:
df = spark.read.parquet(f"{raw_path}/*")
df = df.withColumn("INGEST_DATE", current_timestamp())
df = df.withColumn("DATA_EXTRACAO", to_utc_timestamp(col("DATA_EXTRACAO"), "America/Sao_Paulo"))

In [0]:
(df.write
  .format("delta")
  .option("overwriteSchema", "true")
  .option("mergeSchema", "true")
  .option("path", std_path)
  .mode("overwrite")
  .saveAsTable(dbTable))

print("Dados Carregados")